# Explore Model Registry Content

This notebook is meant to explore the content of the model registry. Such as:
- Listing all available models and their corresponding info
- Listing all available versions and info about a specific model.

### Please duplicate the notebook to start your work and keep the template intact.

## Before running the notebook
The trial datastore process must be started prior to running the notebook. Make sure you open a terminal, activate the .venv environment, run:
```
python -m launch_local_services
```

Once you are done with working in this notebook, close the terminal window.

## Cogment Setup

In [1]:
import os
import sys

ROOT_DIR = os.path.abspath(os.path.join(os.getcwd()))
sys.path.append(ROOT_DIR)

from cogment_verse.utils.generate import generate
from cogment_verse.app import SPEC_FILEPATH

WORK_DIR = os.path.join(ROOT_DIR, ".cogment_verse")

generate(WORK_DIR, SPEC_FILEPATH)

In [3]:
import asyncio
import functools
import re
import json
from dataclasses import dataclass, field
from datetime import datetime
from typing import List

import cogment
import numpy as np
import pandas as pd
from cogment.control import TrialState
from tabulate import tabulate

from cogment_verse.specs import (
    AgentConfig,
    cog_settings,
    EnvironmentConfig,
    EnvironmentSpecs,
    HUMAN_ACTOR_IMPL,
    PLAYER_ACTOR_CLASS,
    TEACHER_ACTOR_CLASS,
    WEB_ACTOR_NAME,
)
from cogment_verse.specs.ndarray_serialization import deserialize_ndarray, serialize_ndarray
from cogment_verse.services_directory import ServiceDirectory,ServiceType
from cogment_verse.model_registry import ModelRegistry

MODEL_REGISTRY_ENDPOINT = "grpc://localhost:9002"

service_directory = ServiceDirectory()
service_directory.add(
    service_type=ServiceType.MODEL_REGISTRY,
    service_endpoint=MODEL_REGISTRY_ENDPOINT,
)
model_registry = ModelRegistry(service_directory)

In [6]:
import logging
import os
import subprocess
import time

import hydra
import pytest
from hydra import compose, initialize
from hydra.core.hydra_config import HydraConfig
from hydra.core.global_hydra import GlobalHydra
from omegaconf import OmegaConf

import cogment_verse
from cogment_verse.constants import CONFIG_DIR, DEFAULT_CONFIG_NAME, DEFAULT_WORK_DIR, TEST_DIR
from cogment_verse.processes.popen_process import PopenProcess
from cogment_verse.services_directory import ServiceType


log = logging.getLogger(__name__)


def on_cra_log(_name, log_line):
    log.info(log_line)

@hydra.main(version_base=None, config_path=CONFIG_DIR, config_name=DEFAULT_CONFIG_NAME)
def test_simple_dqn_cartpole():

    # GlobalHydra.instance().clear()
    # GlobalHydra.instance().initialize(version_base=None, config_path=CONFIG_REL_PATH, job_name="functional_test")
    # cfg = compose(config_name=DEFAULT_CONFIG_NAME)
    # print(OmegaConf.to_yaml(cfg))

  # PopenProcess(
  #     name="test",
  #     args=["HYDRA_FULL_ERROR=1", "python", "-m", "main", "+experiment=simple_dqn/cartpole"],
  #     on_log=on_cra_log
  # )
  process = subprocess.run(args=["python", "-m", "main", "+experiment=simple_dqn/cartpole"])



test_simple_dqn_cartpole()



usage: ipykernel_launcher.py [--help] [--hydra-help] [--version]
                             [--cfg {job,hydra,all}] [--resolve]
                             [--package PACKAGE] [--run] [--multirun]
                             [--shell-completion] [--config-path CONFIG_PATH]
                             [--config-name CONFIG_NAME]
                             [--config-dir CONFIG_DIR]
                             [--experimental-rerun EXPERIMENTAL_RERUN]
                             [--info [{all,config,defaults,defaults-tree,plugins,searchpath}]]
                             [overrides ...]
ipykernel_launcher.py: error: unrecognized arguments: -f


SystemExit: 2

/home/william/dev/cogment-verse/.venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3468: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## Retrieve all model info

In [3]:
model_infos = await model_registry.list_models()

print("*** ALL AVAILABLE MODELS ***")
for model_info in model_infos:
    print(json.dumps(model_info, indent=4))

*** ALL AVAILABLE MODELS ***


## Retrieve version info about specific model

In [24]:
model_id = "tender_lamport_0_model"
version_infos = await model_registry.list_model_versions(model_id)

print(f"Versions available: {len(version_infos)+1}")
print("*** ONLY PRINTING THE 3 LATEST VERSIONS ***")
for version in version_infos[-3:]:
    print(json.dumps(version, indent=4))

Versions available: 20
*** ONLY PRINTING THE 3 LATEST VERSIONS ***
{
    "model_id": "tender_lamport_0_model",
    "version_number": 18,
    "creation_timestamp": "1675293570602968228",
    "archived": true,
    "data_hash": "D7ELnjsasTVa0mBRekuGHe6lBKeCcoIQl8vSrroEiWw=",
    "data_size": "26613",
    "user_data": {
        "total_samples": "0"
    }
}
{
    "model_id": "tender_lamport_0_model",
    "version_number": 19,
    "creation_timestamp": "1675293579883061441",
    "archived": true,
    "data_hash": "XYdS/k9POjYmUfydMgSUXC6yOIUvOOakUg5hkZmtrFA=",
    "data_size": "26613",
    "user_data": {
        "total_samples": "0"
    }
}
{
    "model_id": "tender_lamport_0_model",
    "version_number": 20,
    "creation_timestamp": "1675293580245390754",
    "archived": true,
    "data_hash": "BTjbDqM/W+KCMD+163Gye9oadMqLaVHFTnruwpgN0Rs=",
    "data_size": "26613",
    "user_data": {
        "total_samples": "0"
    }
}
